## Initialize Cleaning

### Import Packages and Raw_data

In [71]:
import pandas as pd
from functools import wraps
import datetime as dt
import numpy as np

### Import File

In [72]:
file_name = "2022"

init_df = pd.read_csv(f"../../data/raw/{file_name}.csv.zip", compression="zip")
init_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33057 entries, 0 to 33056
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          33057 non-null  int64 
 1   id                  33057 non-null  object
 2   type                33057 non-null  object
 3   sectionId           33057 non-null  object
 4   sectionName         33057 non-null  object
 5   webPublicationDate  33057 non-null  object
 6   webTitle            33057 non-null  object
 7   webUrl              33057 non-null  object
 8   apiUrl              33057 non-null  object
 9   fields              33057 non-null  object
 10  tags                33057 non-null  object
 11  rights              33057 non-null  object
 12  section             33039 non-null  object
 13  isHosted            33057 non-null  bool  
 14  pillarId            32812 non-null  object
 15  pillarName          32812 non-null  object
dtypes: bool(1), int64(1), 

### Logging

In [73]:
def log_step(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        tic = dt.datetime.now()
        result = func(*args, **kwargs)
        time_taken = str(dt.datetime.now() - tic)
        print(f"{func.__name__}:\n shape={result.shape} took {time_taken}s\n")
        return result

    return wrapper

## Pipeline - Functions

### Start Pipeline

In [74]:
@log_step
def init_pipeline(df):
    return df.copy()

### Unfold Columns

In [75]:
@log_step
def unfold_columns(df):
    dict_cols = ["fields", "rights"] 
    for col in dict_cols:
        new_df = pd.DataFrame()
        new_df[col] = df[col].apply(lambda x: eval(x))
        add_cols_df = pd.json_normalize(new_df[col])
        df = pd.concat([df, add_cols_df], axis=1)
    
    # Tags extraction
    
    df["tags"] = df["tags"].apply(lambda x: eval(x))
    df['tagWebTitle'] = df['tags'].map(lambda x:[i['webTitle'] for i in x])
    df['tagId'] = df['tags'].map(lambda x:[i['id'] for i in x])
    df = df.drop(columns="tags")
    return df.drop(columns=dict_cols)

### Remove duplicates

In [76]:
@log_step
def remove_duplicates(df):
        return df.drop_duplicates()

### Missing Values

In [90]:
@log_step
def missing_values(df):
    df = df.replace("NaN", np.nan)
    print(df.loc[:, df.isna().any()])
    df["starRating"] = df["starRating"].fillna(99)
    df["pillarName"] = df["pillarName"].fillna("None")
    df["commentable"] = df["commentable"].fillna(False)
    df["firstPublicationDate"] = df["firstPublicationDate"].apply(lambda x: x["webPublicationDate"] if x == np.nan else x)

    return df

### Adjust data types

In [78]:
@log_step
def adjust_data_types(df):
       
   # Strings
   df["id"] = df["id"].astype("string")
   df["sectionName"] = df["sectionName"].astype("string")
   df["type"] = df["type"].astype("string")
   df["webTitle"] = df["webTitle"].astype("string")
   df["pillarName"] = df["pillarName"].astype("string")
   df["headline"] = df["headline"].astype("string")
   df["bodyText"] = df["bodyText"].astype("string")
   
   # Numerical
   df["charCount"] = df["charCount"].astype("int")


   #Categorical
   df["productionOffice"] = df["productionOffice"].astype("category")
   

   # boolean
   df["isLive"] = df["isLive"].map({"true": True, "false": False})
   df["syndicatable"] = df["syndicatable"].map({"true": True, "false": False})
   df["commentable"] = df["commentable"].map({"true": True, np.nan: False})
   
   # Datetimes
   df["webPublicationDate"] = pd.to_datetime(df["webPublicationDate"])
   df["firstPublicationDate"] = pd.to_datetime(df["firstPublicationDate"])
   
   return df

### Drop Columns

In [79]:
@log_step
def drop_columns(df):
    cols =[
        "trailText",
        "main",
        "body",
        "webUrl",
        "apiUrl",
        "standfirst",
        "section",
        #"tags",
        "newspaperPageNumber",
        "newspaperEditionDate",
        "byline",
        "pillarId",
        "bylineHtml",
        "displayHint",
        "commentCloseDate",
        "isInappropriateForSponsorship",
        "isPremoderated",
        "liveBloggingNow",
        "publication",
        "shouldHideAdverts",
        "shortUrl",
        "showInRelatedContent",
        "thumbnail",
        "shouldHideReaderRevenue",
        "lastModified",
        "legallySensitive",
        "lang",
        "showAffiliateLinks",
        "wordcount",
        "sensitive",
        "sectionId",
        "subscriptionDatabases",
        "developerCommunity",
        "contributorBio",
        "scheduledPublicationDate"
        ]
    return df.drop(columns=cols)


### CSV

In [80]:
@log_step
def create_csv(df):
    df.to_csv(f"../../data/clean/{file_name}_clean.csv.zip", index=False, compression="zip")
    df.info()
    return df

## Run Cleaning

In [81]:
articles = (
init_df
    .pipe(init_pipeline)       
    .pipe(remove_duplicates)    
    .pipe(unfold_columns)       
    .pipe(drop_columns)          
    .pipe(adjust_data_types)    
    .pipe(missing_values)    
    .pipe(create_csv)      
)


init_pipeline:
 shape=(33057, 16) took 0:00:00.007755s

remove_duplicates:
 shape=(33057, 16) took 0:00:02.849529s

unfold_columns:
 shape=(33057, 53) took 0:00:27.608571s

drop_columns:
 shape=(33057, 19) took 0:00:00.011201s

adjust_data_types:
 shape=(33057, 19) took 0:00:00.146989s

      pillarName      firstPublicationDate commentable starRating
0          Sport 2022-06-02 09:24:35+00:00       False        NaN
1           News 2022-06-02 13:05:44+00:00       False        NaN
2          Sport 2022-06-02 12:13:00+00:00       False        NaN
3           News 2022-06-02 13:35:35+00:00       False        NaN
4           News 2022-06-02 13:21:22+00:00       False        NaN
...          ...                       ...         ...        ...
33052       News 2022-01-01 00:01:33+00:00       False        NaN
33053      Sport 2022-01-01 00:00:33+00:00        True        NaN
33054  Lifestyle 2022-01-01 00:00:33+00:00         NaN        NaN
33055  Lifestyle 2022-01-01 00:00:33+00:00        Tr

In [82]:
articles.isna().sum()

Unnamed: 0              0
id                      0
type                    0
sectionName             0
webPublicationDate      0
webTitle                0
isHosted                0
pillarName              0
headline                0
firstPublicationDate    4
productionOffice        0
isLive                  0
bodyText                0
charCount               0
commentable             0
starRating              0
syndicatable            0
tagWebTitle             0
tagId                   0
dtype: int64

In [83]:
articles.sample(20)

,Unnamed: 0,id,type,sectionName,webPublicationDate,webTitle,isHosted,pillarName,headline,firstPublicationDate,productionOffice,isLive,bodyText,charCount,commentable,starRating,syndicatable,tagWebTitle,tagId
7659,7659,sport/2022/apr/29/nrl-threatens-to-take-grand-...,article,Sport,2022-04-29 06:25:09+00:00,NRL threatens to take grand final away from Sy...,False,Sport,NRL threatens to take grand final away from Sy...,2022-04-29 00:36:44+00:00,AUS,True,"The New South Wales premier, Dominic Perrottet...",4080,False,99,True,"[NRL, Rugby league, Australia sport, Sport, Sy...","[sport/nrl, sport/rugbyleague, sport/australia..."
4949,4949,business/2022/may/11/lloyds-of-london-attempts...,article,Business,2022-05-11 12:16:24+00:00,Lloyd’s of London attempts to shift AGM onlin...,False,News,Lloyd’s of London attempts to shift AGM onlin...,2022-05-11 12:16:24+00:00,UK,True,Insurance market Lloyd’s of London is urging m...,2854,False,99,True,"[Lloyd's, Extinction Rebellion, Protest, Clima...","[business/lloydsoflondon, environment/extincti..."
9682,9682,film/2022/apr/20/the-witch-robert-eggers-folk-...,article,Film,2022-04-19 17:30:07+00:00,The Witch: Robert Eggers’ folk horror debut wo...,False,Arts,The Witch: Robert Eggers’ folk horror debut wo...,2022-04-19 17:30:07+00:00,AUS,True,"Robert Eggers’ new movie, The Northman, arrive...",3479,True,99,True,"[Horror films, Film, Culture, Stream team, Any...","[film/horror, film/film, culture/culture, cult..."
7750,7750,commentisfree/2022/apr/28/the-guardian-view-on...,article,Opinion,2022-04-28 17:27:57+00:00,The Guardian view on the parliamentary session...,False,Opinion,The Guardian view on the parliamentary session...,2022-04-28 17:27:57+00:00,UK,True,At lunchtime on Thursday parliament was prorog...,3728,False,99,True,"[Opinion, Conservatives, Politics, Labour, Sex...","[commentisfree/commentisfree, politics/conserv..."
9897,9897,politics/2022/apr/18/no-10-suspected-of-being-...,article,Politics,2022-04-18 16:26:51+00:00,No 10 suspected of being target of NSO spyware...,False,News,No 10 suspected of being target of NSO spyware...,2022-04-18 14:38:38+00:00,UK,True,Boris Johnson has been told his Downing Street...,5759,False,99,True,"[Boris Johnson, Politics, UK news, Surveillanc...","[politics/boris-johnson, politics/politics, uk..."
19459,19459,food/2022/mar/05/tomato-blood-orange-salad-wit...,article,Food,2022-03-05 10:30:21+00:00,Ixta Belfrage’s vegan recipe for tomato and bl...,False,Lifestyle,Ixta Belfrage’s vegan recipe for tomato and bl...,2022-03-05 10:30:21+00:00,UK,True,I first developed this recipe for a pop-up at ...,3198,True,99,True,"[The new vegan, Side dishes, Food, Fruit, Vege...","[food/series/the-new-vegan, food/side-dishes, ..."
19098,19098,world/2022/mar/07/next-stop-sylvia-plath-why-i...,article,World news,2022-03-07 10:00:33+00:00,"Next stop, Sylvia Plath! Why it is time to red...",False,News,"Next stop, Sylvia Plath! Why it is time to red...",2022-03-07 10:00:33+00:00,UK,True,"When I was a baby feminist, I would argue with...",9525,False,99,True,"[International Women's Day, London Underground...","[world/international-womens-day, uk/london-und..."
4854,4854,politics/2022/may/11/michael-gove-havoc-on-bre...,article,Politics,2022-05-11 17:59:16+00:00,Michael Gove causes havoc on breakfast TV – bu...,False,News,Michael Gove causes havoc on breakfast TV – bu...,2022-05-11 17:59:16+00:00,UK,True,As a health warning on the dangers of taking i...,5504,False,99,True,"[The politics sketch, Michael Gove, Breakfast ...","[politics/series/the-politics-sketch, politics..."
9360,9360,science/2022/apr/21/mind-satellite-mission-cle...,article,Science,2022-04-21 09:00:51+00:00,Mind that satellite! The mission to clean up d...,False,News,Mind that satellite! The mission to clean up d...,2022-04-21 09:00:51+00:00,UK,True,"In November last year, the five astronauts and...",13095,True,99,True,"[Space, European Space Agency, International S...","[science/space, science/european-space-agen

In [84]:
articles.commentable

0        False
1        False
2        False
3        False
4        False
         ...  
33052    False
33053     True
33054    False
33055     True
33056     True
Name: commentable, Length: 33057, dtype: bool

In [85]:
test = pd.read_csv(f"../../data/raw/{file_name}.csv.zip", compression="zip")

In [86]:
test["tags"] = test["tags"].apply(lambda x: eval(x))
test['tagWebTitle'] = test['tags'].map(lambda x:[i['webTitle'] for i in x])
test['tagId'] = test['tags'].map(lambda x:[i['id'] for i in x])
test = test.drop(columns="tags")
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33057 entries, 0 to 33056
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          33057 non-null  int64 
 1   id                  33057 non-null  object
 2   type                33057 non-null  object
 3   sectionId           33057 non-null  object
 4   sectionName         33057 non-null  object
 5   webPublicationDate  33057 non-null  object
 6   webTitle            33057 non-null  object
 7   webUrl              33057 non-null  object
 8   apiUrl              33057 non-null  object
 9   fields              33057 non-null  object
 10  rights              33057 non-null  object
 11  section             33039 non-null  object
 12  isHosted            33057 non-null  bool  
 13  pillarId            32812 non-null  object
 14  pillarName          32812 non-null  object
 15  tagWebTitle         33057 non-null  object
 16  tagId               33

In [87]:
test.columns

Index(['Unnamed: 0', 'id', 'type', 'sectionId', 'sectionName',
       'webPublicationDate', 'webTitle', 'webUrl', 'apiUrl', 'fields',
       'rights', 'section', 'isHosted', 'pillarId', 'pillarName',
       'tagWebTitle', 'tagId'],
      dtype='object')

In [88]:
test[["tagWebTitle", "tagId"]]

,tagWebTitle,tagId
0,"[England v New Zealand 2022, Cricket, England ...","[sport/england-v-new-zealand-2022, sport/crick..."
1,"[US politics live with Joan E Greve, US politi...",[us-news/series/us-politics-live-with-joan-e-g...
2,"[French Open 2022, French Open, Iga Swiatek, C...","[sport/french-open-2022, sport/frenchopen, spo..."
3,"[Ukraine, Interpol, Russia, Europe, Arms trade...","[world/ukraine, world/interpol, world/russia, ..."
4,"[Queen's platinum jubilee, The Queen, Monarchy...","[uk-news/queens-platinum-jubilee, uk/queen, uk..."
...,...,...
33052,"[Brexit, Food & drink industry, Economics, Bus...","[politics/eu-referendum, business/fooddrinks, ..."
33053,"[Talking Horses, Horse racing, Horse racing ti...","[sport/series/talking-horses, sport/horse-raci..."
33054,"[Crosswords, Crossword, Prize, Brendan]","[crosswords/crosswords, type/crossword, crossw..."
33055,"[Crosswords, Crossword, Quick]","[crosswords/crosswords, type/crossword, crossw..."


In [89]:
test.drop_duplicates()

TypeError: unhashable type: 'list'